In [1]:
import pickle

In [2]:
training_data = []
with open('training_data.pkl' ,'rb')as f :
    training_data = pickle.load(f)

In [3]:
type(training_data)

list

In [4]:
pip install -U gensim

Requirement already up-to-date: gensim in c:\users\asus\anaconda3\lib\site-packages (4.1.2)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import multiprocessing
from gensim.models import Word2Vec
import time
import numpy as np
from numpy.linalg import norm

In [6]:
cores = multiprocessing.cpu_count()
print('Number of cores in this device {}'.format(cores))

Number of cores in this device 8


In [7]:
docs_num = len(training_data)
tokens_num = sum([len(x) for x in training_data])
print('Number of all docs {}'.format(docs_num))
print('Number of all Tokens {}'.format(tokens_num))

Number of all docs 7562
Number of all Tokens 2890435


In [8]:
%%time
w2v_model = Word2Vec(min_count=1 , window=5 , vector_size=300 , alpha=0.03 , workers=cores-1)
w2v_model.build_vocab(training_data)
w2v_model_vocab_size = len(w2v_model.wv)
print('Vocab size : ' , w2v_model_vocab_size)

Vocab size :  51310
Wall time: 3.91 s


In [55]:
start = time.time()
w2v_model.train(training_data , total_examples=w2v_model.corpus_count , epochs= 20)
end = time.time()
print("{} s". format(end-start))

57.118751764297485 s


In [56]:
w2v_model.save("w2v_300d.model")
# w2v_model = Word2Vec.load("w2v_300d.model")

In [11]:
len(w2v_model.wv)

51310

In [12]:
import pickle
docs_tfidf = []
with open('docs_tfidf.pkl' ,'rb')as f :
    docs_tfidf = pickle.load(f)

In [13]:
docs_embedding = []
for doc in docs_tfidf :
    doc_vector = np.zeros(300)
    weight_sum = 0
    for token , weight in doc.items() :
        doc_vector += w2v_model.wv[token] * weight
        weight_sum += weight
    docs_embedding.append(doc_vector/weight_sum)

In [46]:
def similarity(doc1 , doc2) :
    similarity_score = np.dot(doc1 , doc2) / ((norm(doc1)) *(norm(doc2)) )
    return (similarity_score +1)/2

In [47]:
similarity(docs_embedding[12] , docs_embedding[7])

0.6852303643425763

In [16]:
similarity(docs_embedding[12] , docs_embedding[10])

0.9286343226412004

In [17]:
pip install hazm

Note: you may need to restart the kernel to use updated packages.


In [20]:
import hazm
hazm_normalaizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()
stemmer_hazm = hazm.Stemmer()

In [48]:
from collections import Counter
import math
def query_embeding(query) :
       
        normal_query = hazm_normalaizer.normalize(query)
        tokens = hazm.word_tokenize(normal_query)
        tokens = [stemmer_hazm.stem(t) for t in tokens]
        token_tfidf = {}
#         query_embedding  = []
        
        for term, count in Counter(tokens).items():
            tf = (1 + math.log10(count))
            idf = math.log10(7562/1)
            tfidf = tf * idf
            token_tfidf [term] = tfidf
        
        query_vector = np.zeros(300)
        weight_sum = 0
        for token , weight in token_tfidf.items() :
            if (w2v_model.wv[token].nonzero) :
                query_vector += w2v_model.wv[token] * weight
                weight_sum += weight
#         query_embedding.append(query_vector/weight_sum)   
     
        return query_vector/weight_sum
    

In [22]:
len(docs_embedding)

7562

In [52]:
import xlrd
data_set = 'IR1_7k_news.xlsx'
data_reader = xlrd.open_workbook(data_set)
content = data_reader.sheet_by_index(0)

In [53]:
def answer_query(query , k):
    answers = {}
    query_embed = query_embeding(query)
    for i in range(len(docs_embedding)):
      score = similarity(docs_embedding[i],query_embed)
      answers[i] = score
    answers = dict(sorted(answers.items(), key=lambda item: item[1], reverse=True))
    k_first = dict(list(answers.items())[0: k])
#     print(k_first)
    for d in k_first:
        print("{} : {}".format(d,k_first[d]))
        title = data_reader.sheet_by_index(0).cell(int(d), 2).value
        url =  data_reader.sheet_by_index(0).cell(int(d), 1).value
        print(title)
        print(url)
        print("================================================")
    
    

In [58]:
query = input("Enter your query")
answer_query(query,10)

Enter your queryواکسن آسترازنکا
5913 : 0.780641327390907
سالار عقیلی با «نفس بریده» هم صدای مردم شد + ویدئو
https://www.isna.ir/news/1400060201597/سالار-عقیلی-با-نفس-بریده-هم-صدای-مردم-شد-ویدئو
5821 : 0.7773474751710941
نتایج اولیه آزمون دکتری وزارت بهداشت اعلام شد/اعلام زمان انتخاب رشته داوطلبان
https://www.isna.ir/news/1400051309410/نتایج-اولیه-آزمون-دکتری-وزارت-بهداشت-اعلام-شد-اعلام-زمان-انتخاب
5828 : 0.7773474751710941
نتایج اولیه آزمون دکتری وزارت بهداشت اعلام شد/اعلام زمان انتخاب رشته داوطلبان
https://www.isna.ir/news/1400051309410/نتایج-اولیه-آزمون-دکتری-وزارت-بهداشت-اعلام-شد-اعلام-زمان-انتخاب
5970 : 0.772101210756347
رکوردشکنی دوباره تزریق واکسن کرونا در ایران/ تزریق ۱۱۱۲۴۴۰ دُز در شبانه روز گذشته
https://www.isna.ir/news/1400061712949/رکوردشکنی-دوباره-تزریق-واکسن-کرونا-در-ایران-تزریق-۱۱۱۲۴۴۰-دز
5955 : 0.7566129744896292
هجدهمین کنگره بین‌المللی ام.اس ایران برگزار می‌شود
https://www.isna.ir/news/1400061410461/هجدهمین-کنگره-بین-المللی-ام-اس-ایران-برگزار-می-شود
5927 : 0.756134161